In [1]:
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier

from ssri_interactions.io import load_derived_generic
from ssri_interactions.config import ExperimentInfo, Config
from ssri_interactions.decoding.loaders import FSDecodeDataLoader, FSFastDecodeDataLoaderTwoWindows
from ssri_interactions.decoding.preprocessors import DecodePreprocessor
from ssri_interactions.decoding.runners import DecodeRunner
from ssri_interactions.decoding.decoders import Decoder
from ssri_interactions.decoding.shuffle import shuffle_X

/opt/miniconda3/envs/ssri/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
def make_fast_ts_runner(window_1, window_2, bin_width=0.1):
    loader = FSFastDecodeDataLoaderTwoWindows(
        session_name=None, 
        bin_width=bin_width, 
        window_1=window_1,
        window_2=window_2,
    )
    preprocessor = DecodePreprocessor(
        thresh_empty=1, 
    )
    spikes, states = loader()
    spikes, states = preprocessor(spikes, states)
    estimator = LGBMClassifier(n_estimators=40, max_depth=8, num_leaves=30)
    cv = KFold(n_splits=5, shuffle=True)
    decoder = Decoder(estimator=estimator, cv=cv, shuffler=shuffle_X)
    runner = DecodeRunner(
        loader=loader,
        preprocessor=preprocessor,
        decoder=decoder,
        nboot=75,
    )
    return runner

In [3]:
neuron_types = load_derived_generic("neuron_types.csv").query("group in ('CIT', 'SAL')")
sessions = neuron_types.query("experiment_name == 'HAMILTON'").session_name.unique().tolist()
session = sessions[0]

#### Initial window versus baseline

In [4]:
window_pre = (-0.5, -0.1)
window_1 = (0.05, 0.2)
window_2 = (0.5, 0.8)


runner_w1 = make_fast_ts_runner(
    window_1=window_pre,
    window_2=window_1, 
    bin_width=0.1
)
pop_w1, unit_w1 = runner_w1.run_multiple(sessions)


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2079 but corresponding boolean dimension is 183386511

#### Second window versus baseline

In [ ]:
runner_w2 = make_fast_ts_runner(
    window_1=window_pre,
    window_2=window_2, 
    bin_width=0.1
)
pop_w2, unit_w2 = runner_w2.run_multiple(sessions)

#### Initial window versus second window

In [ ]:
runner_comp = make_fast_ts_runner(
    window_1=window_1,
    window_2=window_2, 
    bin_width=0.1
)
pop_comp, unit_comp = runner_comp.run_multiple(sessions)